# Week 3: Exploring and understanding data

Today we're going to use a tabular dataset to get hands-on experience with two other core Python data science libraries: pandas and matplotlib. We're going to use these libraries to explore, do statistics on, and visualize different parts of our dataset to get a handle on what's there.

Outline:
- What is tabular data?
- Intro to pandas
    - Basics
    - Series and dataframe
    - Load a csv
    - Use head, tail, sample, info, describe, shape,  to look at what's in the dataset
    - Your turn: do some stuff with the dataset, some series, etc
- Dealing with missing data
    - count_na, fill_na
    - Imputing missing values: what's the best way of doing it?
- More exploratory data analysis
    - intro to probability distributions - use dataset to demonstrate principles of:
        - what makes a probability distribution?
        - discrete vs continuous
        - joint and conditional probabilities
        - your turn: estimate some probabilities from dataset
    - value_counts - how can we use this to distinguish between continuous and discrete?
    - corr
    - skew
- Visualizing our data
    - Intro to matplotlib
    - Histograms, scatter plots
    - your turn: make some interesting plot
- Is there some linear algebra we can do this week to reinforce the concepts from last week?

## Homework review

## Intro: what is tabular data?

The tabular format is fundamental to data science. We got a taste of tabular data last week when we loaded CSV and JSON data, but let's take a closer look now.

Most people are familiar with tabular data from working with spreadsheet software like Excel. In a table, "records" or "samples" are stored in rows, and "features" or "attributes" are stored in columns. For example, in the `good_movies.csv` dataset that we took a look at last week, there were 7 columns representing the fields `title`, `year`, `oscar_nominations`, `short_summary`, `star_1`, `star_2`, and `star_3` and 4 rows representing the movies *La La Land*, *Moonlight*, *Argo*, and *Gone Girl*.

![](images/good_movies_table.png)

In other words, the rows are individual movies, and the columns represent pieces of information that we know about each movie.

## Pandas

Pandas is the Swiss Army Knife of data analysis in Python. Built on top of NumPy, Pandas wraps arrays with additional functions and metadata to create data frames, a paradigm for storing tabular data borrowed from R.

Let's use Pandas to read and explore the `good_movies.csv` dataset again:

In [1]:
import pandas as pd

good_movies = pd.read_csv('data/good_movies.csv')

In [5]:
good_movies.head()

,title,year,oscar_nominations,short_summary,star_1,star_2,star_3
0,La La Land,2016,14,A jazz pianist falls for an apsiring actres in...,Ryan Gosling,Emma Stone,Rosemarie DeWitt
1,Moonlight,2016,8,A timeless story of human self-discovery and c...,Mahershala Ali,Sheriff Earp,Duan Sanderson
2,Argo,2012,3,Acting under the cover of a Hollywood producer...,Ben Affleck,Bryan Cranston,John Goodman
3,Gone Girl,2014,1,With his wife's disappearance having become th...,Ben Affleck,Rosamund Pike,Neil Patrick Harris


Pandas automatically gives each row an integer index that guarantees the row can be uniquely identified, but otherwise, the data is exactly the same. The `good_movies.head()` method prints out a few rows from the "head" (top) of the dataset. Since there were only 4 rows in this dataset, `head` prints them all.

How exactly does Pandas store the data?

In [6]:
type(good_movies)

pandas.core.frame.DataFrame

One of the fundamental data structures in Pandas is the DataFrame, which stores 2-dimensional (i.e. tabular/matrix) data. There are a few core methods for understanding DataFrames that will be important to understand. We already saw `head()` for print the first several rows of a DataFrame. Some others are:

In [10]:
good_movies.shape    # dataset has 4 rows and 7 columns

(4, 7)

In [11]:
good_movies.info()    # tell me some basic info about what's in each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
title                4 non-null object
year                 4 non-null int64
oscar_nominations    4 non-null int64
short_summary        4 non-null object
star_1               4 non-null object
star_2               4 non-null object
star_3               4 non-null object
dtypes: int64(2), object(5)
memory usage: 304.0+ bytes


In [13]:
good_movies.describe()    # give some summary statistics for the numeric columns

,year,oscar_nominations
count,4.000000,4.000000
mean,2014.500000,6.500000
std,1.914854,5.802298
min,2012.000000,1.000000
25%,2013.500000,2.500000
50%,2015.000000,5.500000
75%,2016.000000,9.500000
max,2016.000000,14.000000


Indexing Pandas DataFrames is a bit different than NumPy. In particular, you can index by the *name* of rows and columns with `loc` or by their *index* with `iloc`. For example, if we wanted to see the summary of *Gone Girl*, we could use:

In [26]:
gone_girl_summary = good_movies.loc[3, 'short_summary']
print(gone_girl_summary)

With his wife's disappearance having become the focus of an intense media circus, a man sees the spotlight turned on him when it's suspected that he may not be innocent.


In [27]:
gone_girl_summary = good_movies.iloc[3, 3]
print(gone_girl_summary)

With his wife's disappearance having become the focus of an intense media circus, a man sees the spotlight turned on him when it's suspected that he may not be innocent.


**Question**: Why did the first value in the index argument not change?

Or, instead, we can retrieve *all* of the summaries:

In [24]:
summaries = good_movies.loc[:, 'short_summary']
print(summaries)

0    A jazz pianist falls for an apsiring actres in...
1    A timeless story of human self-discovery and c...
2    Acting under the cover of a Hollywood producer...
3    With his wife's disappearance having become th...
Name: short_summary, dtype: object


In [25]:
summaries = good_movies.iloc[:, 3]    # short_summaries is the third column
print(summaries)

0    A jazz pianist falls for an apsiring actres in...
1    A timeless story of human self-discovery and c...
2    Acting under the cover of a Hollywood producer...
3    With his wife's disappearance having become th...
Name: short_summary, dtype: object


Speaking of columns, how are these columns from our dataset being stored after we extract them from our original DataFrame?

In [28]:
type(summaries)

pandas.core.series.Series

The `Series` object is another fundamental data type in Pandas. `Series` objects store 1-dimensional (i.e. vector) data, like a single column of a `DataFrame`.

For dealing with tabular data, `Series` and `DataFrames` are much more powerful than pure NumPy arrays. For example, we're not forced to index rows by integers; we can specify a column (as long as it contains unique elements) to use as an index:

In [32]:
good_movies = pd.read_csv('data/good_movies.csv', index_col='title')
good_movies.head()

,year,oscar_nominations,short_summary,star_1,star_2,star_3
title,,,,,,
La La Land,2016,14,A jazz pianist falls for an apsiring actres in...,Ryan Gosling,Emma Stone,Rosemarie DeWitt
Moonlight,2016,8,A timeless story of human self-discovery and c...,Mahershala Ali,Sheriff Earp,Duan Sanderson
Argo,2012,3,Acting under the cover of a Hollywood producer...,Ben Affleck,Bryan Cranston,John Goodman
Gone Girl,2014,1,With his wife's disappearance having become th...,Ben Affleck,Rosamund Pike,Neil Patrick Harris


In [33]:
good_movies.loc['Gone Girl', 'short_summary']

"With his wife's disappearance having become the focus of an intense media circus, a man sees the spotlight turned on him when it's suspected that he may not be innocent."

In [34]:
summaries = good_movies.loc[:, 'short_summary']
summaries.loc['Gone Girl']

"With his wife's disappearance having become the focus of an intense media circus, a man sees the spotlight turned on him when it's suspected that he may not be innocent."

It's also easy to filter rows on certain conditions:

In [35]:
good_movies[good_movies['oscar_nominations'] > 5]

,year,oscar_nominations,short_summary,star_1,star_2,star_3
title,,,,,,
La La Land,2016,14,A jazz pianist falls for an apsiring actres in...,Ryan Gosling,Emma Stone,Rosemarie DeWitt
Moonlight,2016,8,A timeless story of human self-discovery and c...,Mahershala Ali,Sheriff Earp,Duan Sanderson


It's also very convenient to do arithmetic and summary statistics on the data:

In [38]:
good_movies['oscar_nominations'].count()

4

In [39]:
good_movies['oscar_nominations'].sum()

26

In [40]:
good_movies['oscar_nominations'].mean()

6.5

In [41]:
good_movies['oscar_nominations'].median()

5.5

In [43]:
good_movies['oscar_nominations'].std()

5.802298395176403

Fundamental structures: Series and DataFrame
    
Indexing - by name and by location

Reading in a csv and exploring it

Your turn - read csv, get number of rows and columns, get column data types, do something to one of the columns